In [1]:
import random
import pennylane as qml
from pennylane import numpy as np
from maskit.datasets import load_data

In [2]:
# Setting seeds for reproducible results
np.random.seed(1337)
random.seed(1337)

# Loading the data

Data of interest is MNIST data. As we want to go for reproducible results, we
will first go with the option `shuffle=False`. For the rest of the parameters,
we now go with the default options. This gives us data for two classes, the
written numbers 6 and 9. We also only get a limited number of sampes, that is
100 samples for training and 50 for testing. For further details see the
appropriate docstring.

In [3]:
data = load_data("mnist", shuffle=False, target_length=2)

2021-10-28 20:01:26.522031: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Setting up a Variational Quantum Circuit for training

There is an example on the [PennyLane website](https://pennylane.ai/qml/demos/tutorial_variational_classifier.html#iris-classification) for iris data showing a setup for a variational classifier. That is variational quantum circuits that can be trained from labelled (classical) data.

In [4]:
wires = 4
layers = 4
epochs = 5
parameters = np.random.uniform(low=-np.pi, high=np.pi, size=(layers, wires, 2))

In [5]:
def variational_circuit(params):
    for layer in range(layers):
        for wire in range(wires):
            qml.RX(params[layer][wire][0], wires=wire)
            qml.RY(params[layer][wire][1], wires=wire)
        for wire in range(0, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
        for wire in range(1, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
    return qml.expval(qml.PauliZ(0))

In [6]:
def variational_training_circuit(params, data):
    qml.templates.embeddings.AngleEmbedding(
        features=data, wires=range(wires), rotation="X"
    )
    return variational_circuit(params)

In [7]:
dev = qml.device('default.qubit', wires=wires, shots=1000)
circuit = qml.QNode(func=variational_circuit, device=dev)
training_circuit = qml.QNode(func=variational_training_circuit, device=dev)

In [8]:
circuit(parameters)

tensor(-0.052, requires_grad=True)

In [9]:
training_circuit(parameters, data.train_data[0])

tensor(0.014, requires_grad=True)

In [10]:
print(training_circuit.draw())

 0: ──RX(0.105)──RX(-1.5)───RY(-2.14)───╭C───RX(1.46)────RY(-2.42)──────────────╭C───RX(1.85)───RY(-0.872)─────────────╭C───RX(-0.00221)──RY(-2.02)──────────────╭C──────┤ ⟨Z⟩ 
 1: ──RX(2.88)───RX(-1.39)──RY(-0.256)──╰Z──╭C───────────RX(-0.715)──RY(0.807)──╰Z──╭C──────────RX(-0.527)──RY(0.529)──╰Z──╭C─────────────RX(-0.546)──RY(-1.89)──╰Z──╭C──┤     
 2: ──RX(1.85)───RX(-1.12)──RY(0.116)───╭C──╰Z───────────RX(-2.36)───RY(3.04)───╭C──╰Z──────────RX(1.63)────RY(-1.96)──╭C──╰Z─────────────RX(0.199)───RY(2.09)───╭C──╰Z──┤     
 3: ──RX(0.862)──RX(-1.5)───RY(2.99)────╰Z───RX(-0.357)──RY(1.82)───────────────╰Z───RX(-1.33)──RY(1.07)───────────────╰Z───RX(-1.98)─────RY(2.87)───────────────╰Z──────┤     



In [11]:
# some helpers
def correctly_classified(params, data, target):
    prediction = training_circuit(params, data)
    if prediction < 0 and target[0] > 0:
        return True
    elif prediction > 0 and target[1] > 0:
        return True
    return False

def overall_cost_and_correct(cost_fn, params, data, targets):
    cost = correct_count = 0
    for datum, target in zip(data, targets):
        cost += cost_fn(params, datum, target)
        correct_count += int(correctly_classified(params, datum, target))
    return cost, correct_count

In [12]:
# Playing with different cost functions
def crossentropy_cost(params, data, target):
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return cross_entropy(predictions, target)

def distributed_cost(params, data, target):
    """Cost function distributes probabilities to both classes."""
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return np.sum(np.abs(target - predictions))

def cost(params, data, target):
    """Cost function penalizes choosing wrong class."""
    prediction = training_circuit(params, data)
    predictions = np.array([0, prediction]) if prediction > 0 else np.array([prediction * -1, 0])
    return np.sum(np.abs(target - predictions))

In [13]:
optimizer = qml.AdamOptimizer()
cost_fn = cost

In [14]:
start_cost, correct_count = overall_cost_and_correct(cost_fn, parameters, data.test_data, data.test_target)
print(f"start cost: {start_cost}, with {correct_count}/{len(data.test_target)} correct samples")

start cost: 56.230000000000004, with 24/50 correct samples


In [15]:
params = parameters.copy()
for _ in range(epochs):
    for datum, target in zip(data.train_data, data.train_target):
        params = optimizer.step(lambda weights: cost_fn(weights, datum, target), params)

In [16]:
final_cost, correct_count = overall_cost_and_correct(cost_fn, params, data.test_data, data.test_target)
print(f"final cost: {final_cost}, with {correct_count}/{len(data.test_target)} correct samples")

final cost: 31.440000000000005, with 39/50 correct samples
